In [17]:
import mysql.connector


"""
使用该方法将描述存到项目文件中，index by 数据库名
"""


def get_descriptions_from_database(cnx):
    if not cnx:
        return None
    try:
        cursor = cnx.cursor()
       
    
        # 创建一个游标对象
        cursor = cnx.cursor()
        get_table = "SHOW TABLES"
        cursor.execute(get_table)
        tables = cursor.fetchall()
        table_names = [table[0] for table in tables]
        table_descriptions = {}
        for table_name in table_names:
            # 查询表结构信息
            describe_query = f"SHOW FULL COLUMNS FROM {table_name}"
            cursor.execute(describe_query)

            # 获取字段描述信息
            
            column_descriptions = {}
            
            for column in cursor.fetchall():
                column_name = column[0]
                column_comment = column[8]
                if column_comment:
                    column_descriptions[column_name] = column_comment
                
            table_descriptions[table_name] = column_descriptions
        return table_descriptions

    except mysql.connector.Error as err:
        print(f"to database err:{err}")
        return None

    finally:
        # 关闭游标和连接
        cursor.close()
        

# 调用函数查询字段描述
descriptions = get_descriptions_from_database(cnx)

# 打印字段描述
if descriptions:
    for table, columns in descriptions.items():
        print(f"Table {table}:")
        for column, description in columns.items():
            print(f"  Column {column}: {description}")

Table actor:
  Column first_name: first_name
  Column last_name: last_name
  Column last_update: last_update
Table actor_info:
  Column first_name: first_name
  Column last_name: last_name
Table address:
  Column address: address
  Column address2: address2
  Column district: district
  Column city_id: city_id
  Column postal_code: postal_code
  Column phone: phone
  Column last_update: last_update
Table category:
  Column name: name
  Column last_update: last_update
Table city:
  Column city: city
  Column country_id: country_id
  Column last_update: last_update
Table country:
  Column country: country
  Column last_update: last_update
Table customer:
  Column store_id: store_id
  Column first_name: first_name
  Column last_name: last_name
  Column email: email
  Column address_id: address_id
  Column active: active
  Column create_date: create_date
  Column last_update: last_update
Table customer_list:
  Column address: address
  Column zip code: postal_code
  Column phone: phone
  C

In [4]:
# 生成datasource.json

import os
import json

with open('datasource.json', 'r') as f:
    jsons = json.load(f)

names = []
for root, dirs, files in os.walk('./data/4_26/complex/'):
    for file in files:
        tt = file.split('.')[0].split('_')
        tt = '_'.join(tt[:-1])
        names.append(tt)

for name in names:
    if name not in [json['db_name'] for json in jsons]: 
        jsons.append({
            "db_name": name,
            "url": "",
        })

with open('datasource.json', 'w') as f:
    json.dump(jsons, f, indent=4)

In [17]:
# 获取kaggle中关于数据库的description,不是kaggle的手动添加到datasource.json中

import requests
from bs4 import BeautifulSoup
import json

def get_descriptions_from_kaggle():
    with open('datasource.json', 'r') as file:
        urls = json.load(file)

    for item in urls:
        url = item['url']
        if 'kaggle' not in url:
            continue
        db_name = item['db_name']
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        ld_json_tags = soup.find_all('script', type='application/ld+json')
        try:
            tags = json.loads(ld_json_tags[0].text.strip())
        except Exception as e:
            print(f"get description for db{db_name} in {url} error ocur:{e}")
            continue
        descriptions = tags['description']
        item['description'] = descriptions


    with open('datasource.json', 'w') as file:
        json.dump(urls, file, indent=4)

get_descriptions_from_kaggle()

get description for dbhome_credit_default in https://www.kaggle.com/c/home-credit-default-risk/data error ocur:list index out of range
get description for dbprediction in https://www.kaggle.com/c/outbrain-click-prediction/data error ocur:list index out of range


In [20]:
## 手动添加描述


de = """

application_{train|test}.csv

This is the main table, broken into two files for Train (with TARGET) and Test (without TARGET).
Static data for all applications. One row represents one loan in our data sample.
bureau.csv

All client's previous credits provided by other financial institutions that were reported to Credit Bureau (for clients who have a loan in our sample).
For every loan in our sample, there are as many rows as number of credits the client had in Credit Bureau before the application date.
bureau_balance.csv

Monthly balances of previous credits in Credit Bureau.
This table has one row for each month of history of every previous credit reported to Credit Bureau – i.e the table has (#loans in sample * # of relative previous credits * # of months where we have some history observable for the previous credits) rows.
POS_CASH_balance.csv

Monthly balance snapshots of previous POS (point of sales) and cash loans that the applicant had with Home Credit.
This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (#loans in sample * # of relative previous credits * # of months in which we have some history observable for the previous credits) rows.
credit_card_balance.csv

Monthly balance snapshots of previous credit cards that the applicant has with Home Credit.
This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (#loans in sample * # of relative previous credit cards * # of months where we have some history observable for the previous credit card) rows.
previous_application.csv

All previous applications for Home Credit loans of clients who have loans in our sample.
There is one row for each previous application related to loans in our data sample.
installments_payments.csv

Repayment history for the previously disbursed credits in Home Credit related to the loans in our sample.
There is a) one row for every payment that was made plus b) one row each for missed payment.
One row is equivalent to one payment of one installment OR one installment corresponding to one payment of one previous Home Credit credit related to loans in our sample.
HomeCredit_columns_description.csv

This file contains descriptions for the columns in the various data files.

"""

with open('datasource.json', 'r') as file:
        urls = json.load(file)

for item in urls:
    if not item['db_name'] == 'home_credit_default':
        continue
    item['description'] = de

with open('datasource.json', 'w') as file:
        json.dump(urls, file, indent=4)

In [11]:
# 获取sqlite元数据

import sqlite3

def extract_tables_and_columns(db_file):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    tables_info = {}
    for table in tables:
        table_name = table[0]
        try:
            cursor.execute("PRAGMA table_info({});".format(table_name))
        except sqlite3.DatabaseError as e:
            print(f"Error: {e}")
            continue 
        columns_info = cursor.fetchall()
        columns = [(col[1], col[2]) for col in columns_info]
        tables_info[table_name] = columns

    conn.close()

    structure_description = []
    for table_name, columns_info in tables_info.items():
        column_des = []
        for column_name, column_type in columns_info:
            des = f'{column_name}({column_type})'
            column_des.append(des)
        columns_info = ', '.join(column_des)
        table_strucure = f'Table {table_name} contains columns: {columns_info}'
        structure_description.append(table_strucure)

    return tables_info, structure_description

db_file = './database/northwind.sqlite'
tables_info, structure_description = extract_tables_and_columns(db_file)

structure_description 


Error: near "order": syntax error


['Table categories contains columns: CategoryID(INTEGER), CategoryName(VARCHAR(15)), Description(TEXT), Picture(BLOB)',
 'Table sqlite_sequence contains columns: name(), seq()',
 'Table customercustomerdemo contains columns: CustomerID(VARCHAR(5)), CustomerTypeID(VARCHAR(10))',
 'Table customerdemographics contains columns: CustomerTypeID(VARCHAR(10)), CustomerDesc(TEXT)',
 'Table customers contains columns: CustomerID(VARCHAR(5)), CompanyName(VARCHAR(40)), ContactName(VARCHAR(30)), ContactTitle(VARCHAR(30)), Address(VARCHAR(60)), City(VARCHAR(15)), Region(VARCHAR(15)), PostalCode(VARCHAR(10)), Country(VARCHAR(15)), Phone(VARCHAR(24)), Fax(VARCHAR(24))',
 'Table employees contains columns: EmployeeID(INTEGER), LastName(VARCHAR(20)), FirstName(VARCHAR(10)), Title(VARCHAR(30)), TitleOfCourtesy(VARCHAR(25)), BirthDate(DATETIME), HireDate(DATETIME), Address(VARCHAR(60)), City(VARCHAR(15)), Region(VARCHAR(15)), PostalCode(VARCHAR(10)), Country(VARCHAR(15)), HomePhone(VARCHAR(24)), Extension

In [14]:
from src.qa import makecomversation
import yaml
import json
import os
import pandas as pd

def load_config(file_path):
    with open(file_path, 'r') as config_file:
        config = yaml.safe_load(config_file)
    return config
config = load_config('config.yaml')

from openai import APIConnectionError, OpenAI
client = OpenAI(
    api_key=config['api']['key']
)

def get_table_descriptions(structure_info, db_description, table_indicators): ## from gpt
    prompt = f"""You are a database design expert. 
    You need to guess the meaning of the provided fields based on the provided database structure information and a description of the database.
    the database description is:
    {db_description}

    The database structure information is:
    {structure_info}
    """
    usercontent = f"""You need to guess the meaning of each field in the table provided below, and then add a description for each field separately. 
    Make sure there are no omissions.
    
    The table contains the following fields:
    {table_indicators}
    """
    
    return_format = '[{"table indicator": "the meaning of the indicator"}, {"table indicator": "the meaning of the indicator"}, ...]'
    
    result = makecomversation(prompt, usercontent, return_format, client)
    return result


with open('datasource.json', 'r') as file:
        urls = json.load(file)

results = {}
for item in urls:
    if 'description' not in item:
        print(f"database {item['db_name']} has no description, jump to next database")
        print(item)
        continue
    
    db_name = item['db_name']
    print(f"start to process {db_name}")
    db_file = os.path.join('./database/', f'{db_name}.sqlite')
    if not os.path.exists(db_file):
        print(f"database file {db_file} not exists, jump to next database")
        continue
    
    # 如果目录 f'./database_descriptions/' 该f'{db_name}_descriptions.csv' 已经存在了就跳过
    if os.path.exists(f'./database_descriptions/{db_name}_descriptions.csv'):
        print(f"database {db_name} description file already exists, jump to next database")
        continue

    tables_info, structure_description = extract_tables_and_columns(db_file)
    if not tables_info:
        print(f"database {db_name} extract tables and columns failed, jump to next database")
        continue

    table_descriptions = {}
    for table_name in tables_info:
        columns = tables_info[table_name]
        table_indicators = []
        for column in columns:
            indicator = f"{table_name}.{column[0]}"
            table_indicators.append(indicator)
        print(f"start to process table {table_name}, indicators: {table_indicators}")
        table_des = get_table_descriptions(structure_description, item['description'], table_indicators)
        table_descriptions[table_name] = table_des
    
    table_list = []
    column_list = []
    description_list = []

    for table, columns in table_descriptions.items():
        for column, description in columns.items():
            # bug:??
            column = column.split('.')[-1]
            table_list.append(table)
            column_list.append(column)
            description_list.append(description)

    df = pd.DataFrame({
        'table': table_list,
        'column': column_list,
        'description': description_list
    })

    df.to_csv(f'./database_descriptions/{db_name}_descriptions.csv', index=False)
    

start to process airline
database file ./database/airline.sqlite not exists, jump to next database
database airport has no description, jump to next database
{'db_name': 'airport', 'url': 'https://dev.mysql.com/doc/index-other.html'}
database aspartic has no description, jump to next database
{'db_name': 'aspartic', 'url': 'https://wikidbs.github.io/'}
database athletics has no description, jump to next database
{'db_name': 'athletics', 'url': 'https://wikidbs.github.io/'}
database employees has no description, jump to next database
{'db_name': 'employees', 'url': 'https://dev.mysql.com/doc/index-other.html'}
start to process home_credit_default
database home_credit_default description file already exists, jump to next database
database hospital has no description, jump to next database
{'db_name': 'hospital', 'url': 'https://wikidbs.github.io/'}
start to process indicators_2022
database indicators_2022 description file already exists, jump to next database
database italy has no descri

In [ ]:
from src.qa import makecomversation
import yaml
import json
import os
import pandas as pd

def load_config(file_path):
    with open(file_path, 'r') as config_file:
        config = yaml.safe_load(config_file)
    return config
config = load_config('config.yaml')

from openai import APIConnectionError, OpenAI
client = OpenAI(
    api_key=config['api']['key']
)

def get_descriptions(structure_info, db_description, indicators): ## from gpt
    prompt = f"""You are a database design expert. 
    You need to guess the meaning of the provided fields based on the provided database structure information and a description of the database.
    the database description is:
    {db_description}

    The database structure information is:
    {structure_info}
    """
    usercontent = f"""You need to guess the meaning of each field in the table provided below, and then add a description for each field separately. 
    Make sure there are no omissions.
    
    The table contains the following fields:
    {indicators}
    """
    
    return_format = '[{"table indicator": "the meaning of the indicator"}, {"table indicator": "the meaning of the indicator"}, ...]'
    
    result = makecomversation(prompt, usercontent, return_format, client)
    return result


with open('datasource.json', 'r') as file:
        urls = json.load(file)

results = {}
for item in urls:
    if 'description' not in item:
        print(f"database {item['db_name']} has no description, jump to next database")
        print(item)
        continue
    
    db_name = item['db_name']
    print(f"start to process {db_name}")
    db_file = os.path.join('./database/', f'{db_name}.sqlite')
    if not os.path.exists(db_file):
        print(f"database file {db_file} not exists, jump to next database")
        continue
    
    # 如果目录 f'./database_descriptions/' 该f'{db_name}_descriptions.csv' 已经存在了就跳过
    if os.path.exists(f'./database_descriptions/{db_name}_descriptions.csv'):
        print(f"database {db_name} description file already exists, jump to next database")
        continue

    tables_info, structure_description = extract_tables_and_columns(db_file)
    if not tables_info:
        print(f"database {db_name} extract tables and columns failed, jump to next database")
        continue

    table_descriptions = {}
    for table_name in tables_info:
        columns = tables_info[table_name]
        table_indicators = []
        for column in columns:
            indicator = f"{table_name}.{column[0]}"
            table_indicators.append(indicator)
        table_des = get_table_descriptions(structure_description, item['description'], table_indicators)
        table_descriptions[table_name] = table_des
    table_list = []
    column_list = []
    description_list = []

    for table, columns in table_descriptions.items():
        for column, description in columns.items():
            # bug:??
            column = column.split('.')[-1]
            table_list.append(table)
            column_list.append(column)
            description_list.append(description)

    df = pd.DataFrame({
        'table': table_list,
        'column': column_list,
        'description': description_list
    })

    df.to_csv(f'./database_descriptions/{db_name}_descriptions.csv', index=False)
    

In [28]:
# debuging

import re
import pandas as pd


sql = 'SELECT game_summary.natl_tv_broadcaster_abbreviation FROM game_summary AS game_summary ORDER BY game_summary.game_date_est DESC'

# 提取sql相关字段
def get_indicators(clause_list):
  pattern = r'\b\w+\.\w+\b'
  sql_indicators = []
  for clause in clause_list:
    matches = re.findall(pattern, clause)
    sql_indicators += matches
  return sql_indicators # [table.column, table.column, ...]


# 获取字段描述
def get_sql_descriptions(cluse_list, db_name = None):
    if db_name is None:
        return 'no descriptions available, you have to guess from the table name.'
    
    des_path = f'./database_descriptions/{db_name}_descriptions.csv'

    if not os.path.exists(des_path):
        return 'no descriptions available, you have to guess from the table name.'
    
    descriptions = pd.read_csv(des_path)


    indicators = get_indicators(cluse_list)

    relevent_descriptions = []
    for indicator in indicators:
        table, column = indicator.split('.')
        try:
            descirption = descriptions[(descriptions['table'] == table) & (descriptions['column'] == column)]['description'].values[0]
            relevent_descriptions.append(f'column {column} in table {table} is {descirption}. ')
        except KeyError:
            pass

    return relevent_descriptions


sql_descriptions = get_sql_descriptions([sql], 'NBA')

sql_descriptions

['column natl_tv_broadcaster_abbreviation in table game_summary is The abbreviation of the national TV broadcaster',
 'column game_date_est in table game_summary is The date of the game in Eastern Standard Time']

In [7]:
# 遍历4_22 目录下所有文件，将文件名按"_"分割后去掉最后一部分，重命名文件名
import os

for root, dirs, files in os.walk('./data/4_22/complex/'):
    for file in files:
        file_path = os.path.join(root, file)
        if file_path.endswith('.json'):
            file_name = file.split('.')[0]
            new_name = '_'.join(file_name.split('_')[:-1])
            new_path = os.path.join(root, f'{new_name}.json')
            print(f"rename {file_path} to {new_path}")
            os.rename(file_path, new_path)

rename ./data/4_22/complex/indicators_2022_complex.json to ./data/4_22/complex/indicators_2022.json
rename ./data/4_22/complex/matches_football_complex.json to ./data/4_22/complex/matches_football.json
rename ./data/4_22/complex/NBA_complex.json to ./data/4_22/complex/NBA.json
rename ./data/4_22/complex/pitchfork_complex.json to ./data/4_22/complex/pitchfork.json
rename ./data/4_22/complex/spatiaLite_small_complex.json to ./data/4_22/complex/spatiaLite_small.json
rename ./data/4_22/complex/spotify_complex.json to ./data/4_22/complex/spotify.json
rename ./data/4_22/complex/tvmaze_complex.json to ./data/4_22/complex/tvmaze.json
rename ./data/4_22/complex/UK_arms_complex.json to ./data/4_22/complex/UK_arms.json
rename ./data/4_22/complex/WDI_complex.json to ./data/4_22/complex/WDI.json
rename ./data/4_22/complex/yeast_complex.json to ./data/4_22/complex/yeast.json
rename ./data/4_22/complex/yelp_complex.json to ./data/4_22/complex/yelp.json


# 下面是一些测试脚本：

In [24]:
db_name = 'home_credit_default'
db_file = os.path.join('./database/', f'{db_name}.sqlite')
tables_info, structure_description = extract_tables_and_columns(db_file)
table_name = 'application_test'
columns = tables_info[table_name]
table_indicators = []
for column in columns:
    indicator = f"{table_name}.{column[0]}"
    table_indicators.append(indicator)
table_indicators

['application_test.SK_ID_CURR',
 'application_test.NAME_CONTRACT_TYPE',
 'application_test.CODE_GENDER',
 'application_test.FLAG_OWN_CAR',
 'application_test.FLAG_OWN_REALTY',
 'application_test.CNT_CHILDREN',
 'application_test.AMT_INCOME_TOTAL',
 'application_test.AMT_CREDIT',
 'application_test.AMT_ANNUITY',
 'application_test.AMT_GOODS_PRICE',
 'application_test.NAME_TYPE_SUITE',
 'application_test.NAME_INCOME_TYPE',
 'application_test.NAME_EDUCATION_TYPE',
 'application_test.NAME_FAMILY_STATUS',
 'application_test.NAME_HOUSING_TYPE',
 'application_test.REGION_POPULATION_RELATIVE',
 'application_test.DAYS_BIRTH',
 'application_test.DAYS_EMPLOYED',
 'application_test.DAYS_REGISTRATION',
 'application_test.DAYS_ID_PUBLISH',
 'application_test.OWN_CAR_AGE',
 'application_test.FLAG_MOBIL',
 'application_test.FLAG_EMP_PHONE',
 'application_test.FLAG_WORK_PHONE',
 'application_test.FLAG_CONT_MOBILE',
 'application_test.FLAG_PHONE',
 'application_test.FLAG_EMAIL',
 'application_test.OCCUP

In [25]:

db_description = "\n\napplication_{train|test}.csv\n\nThis is the main table, broken into two files for Train (with TARGET) and Test (without TARGET).\nStatic data for all applications. One row represents one loan in our data sample.\nbureau.csv\n\nAll client's previous credits provided by other financial institutions that were reported to Credit Bureau (for clients who have a loan in our sample).\nFor every loan in our sample, there are as many rows as number of credits the client had in Credit Bureau before the application date.\nbureau_balance.csv\n\nMonthly balances of previous credits in Credit Bureau.\nThis table has one row for each month of history of every previous credit reported to Credit Bureau \u2013 i.e the table has (#loans in sample * # of relative previous credits * # of months where we have some history observable for the previous credits) rows.\nPOS_CASH_balance.csv\n\nMonthly balance snapshots of previous POS (point of sales) and cash loans that the applicant had with Home Credit.\nThis table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample \u2013 i.e. the table has (#loans in sample * # of relative previous credits * # of months in which we have some history observable for the previous credits) rows.\ncredit_card_balance.csv\n\nMonthly balance snapshots of previous credit cards that the applicant has with Home Credit.\nThis table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample \u2013 i.e. the table has (#loans in sample * # of relative previous credit cards * # of months where we have some history observable for the previous credit card) rows.\nprevious_application.csv\n\nAll previous applications for Home Credit loans of clients who have loans in our sample.\nThere is one row for each previous application related to loans in our data sample.\ninstallments_payments.csv\n\nRepayment history for the previously disbursed credits in Home Credit related to the loans in our sample.\nThere is a) one row for every payment that was made plus b) one row each for missed payment.\nOne row is equivalent to one payment of one installment OR one installment corresponding to one payment of one previous Home Credit credit related to loans in our sample.\nHomeCredit_columns_description.csv\n\nThis file contains descriptions for the columns in the various data files.\n\n"
table_des = get_table_descriptions(structure_description, db_description, table_indicators)
table_des

{'SK_ID_CURR': 'Unique identifier for the loan application',
 'NAME_CONTRACT_TYPE': 'Type of loan cash or revolving',
 'CODE_GENDER': 'Gender of the applicant',
 'FLAG_OWN_CAR': 'Flag if the applicant owns a car',
 'FLAG_OWN_REALTY': 'Flag if the applicant owns real estate',
 'CNT_CHILDREN': 'Number of children the applicant has',
 'AMT_INCOME_TOTAL': 'Total income of the applicant',
 'AMT_CREDIT': 'Credit amount of the loan',
 'AMT_ANNUITY': 'Loan annuity',
 'AMT_GOODS_PRICE': 'For consumer loans it is the price of the goods for which the loan is given',
 'NAME_TYPE_SUITE': 'Who was accompanying client when he was applying for the loan',
 'NAME_INCOME_TYPE': 'Clients income type (businessman, working, maternity leave, etc.)',
 'NAME_EDUCATION_TYPE': 'Level of highest education the client achieved',
 'NAME_FAMILY_STATUS': 'Family status of the client',
 'NAME_HOUSING_TYPE': 'What is the housing situation of the client (renting, living with parents, etc.)',
 'REGION_POPULATION_RELATIVE'

In [26]:
import pandas as pd

official_des = pd.read_csv('HomeCredit_columns_description.csv')
official_des

,Unnamed: 0,Table,Row,Description,Special,Unnamed: 5,Unnamed: 6
0,1,application_{train|test},SK_ID_CURR,ID of loan in our sample,NaN,NaN,NaN
1,2,application_{train|test},TARGET,Target variable (1 - client with payment diffi...,NaN,NaN,NaN
2,5,application_{train|test},NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN,NaN,NaN
3,6,application_{train|test},CODE_GENDER,Gender of the client,NaN,NaN,NaN
4,7,application_{train|test},FLAG_OWN_CAR,Flag if the client owns a car,NaN,NaN,NaN
...,...,...,...,...,...,...,...
444,120,application_test,AMT_REQ_CREDIT_BUREAU_DAY,Number of enquiries to Credit Bureau about the...,NaN,NaN,NaN
445,121,application_test,AMT_REQ_CREDIT_BUREAU_WEEK,Number of enquiries to Credit Bureau about the...,NaN,NaN,NaN
446,122,application_test,AMT_REQ_CREDIT_BUREAU_MON,Number of enquiries to Credit Bureau about the...,NaN,NaN,NaN
447,123,application_test,AMT_REQ_CREDIT_BUREAU_QRT,Number of enquiries to Credit Bureau about the...,NaN,NaN,NaN


In [31]:
gpt_des = {}
row = []
des = []
for key,value in table_des.items():
    row.append(key)
    des.append(value)
gpt_des['Row'] = row
gpt_des['Description'] = des
gpt_des = pd.DataFrame(gpt_des)
result = pd.merge(gpt_des, official_des, on='Row',how='right')
result

,Row,Description_x,Unnamed: 0,Table,Description_y,Special,Unnamed: 5,Unnamed: 6
0,SK_ID_CURR,Unique identifier for the loan application,1,application_{train|test},ID of loan in our sample,NaN,NaN,NaN
1,TARGET,NaN,2,application_{train|test},Target variable (1 - client with payment diffi...,NaN,NaN,NaN
2,NAME_CONTRACT_TYPE,Type of loan cash or revolving,5,application_{train|test},Identification if loan is cash or revolving,NaN,NaN,NaN
3,CODE_GENDER,Gender of the applicant,6,application_{train|test},Gender of the client,NaN,NaN,NaN
4,FLAG_OWN_CAR,Flag if the applicant owns a car,7,application_{train|test},Flag if the client owns a car,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
444,AMT_REQ_CREDIT_BUREAU_DAY,Number of enquiries to Credit Bureau about the...,120,application_test,Number of enquiries to Credit Bureau about the...,NaN,NaN,NaN
445,AMT_REQ_CREDIT_BUREAU_WEEK,Number of enquiries to Credit Bureau about the...,121,application_test,Number of enquiries to Credit Bureau about the...,NaN,NaN,NaN
446,AMT_REQ_CREDIT_BUREAU_MON,Number of enquiries to Credit Bureau about the...,122,application_test,Number of enquiries to Credit Bureau about the...,NaN,NaN,NaN
447,AMT_REQ_CREDIT_BUREAU_QRT,Number of enquiries to Credit Bureau about the...,123,application_test,Number of enquiries to Credit Bureau about the...,NaN,NaN,NaN


In [35]:
result = result[['Row', 'Description_x', 'Description_y']]
result.columns = ['Row', 'GPT Description', 'Official Description']
result.to_csv('home_credit_default_application_test_gpt_des.csv', index=False)

In [ ]:
import os

def add_create_database_statement(folder_path):
    # 获取指定文件夹下的所有文件
    files = os.listdir(folder_path)
    for file_name in files:
        if file_name.endswith('.sql'):
            file_path = os.path.join(folder_path, file_name)
            # 获取数据库名
            database_name = os.path.splitext(file_name)[0]
            # 创建新文件来保存修改后的内容
            temp_file_path = file_path + '.temp'
            with open(file_path, 'r', encoding='utf-8') as original_file, \
                 open(temp_file_path, 'w', encoding='utf-8') as temp_file:
                first_line = original_file.readline()
                temp_file.write(first_line)
                # 在第一行添加 CREATE DATABASE 语句
                temp_file.write(f"USE `{database_name}`;\n")
                # 复制原文件内容到临时文件
                temp_file.write(original_file.read())
            # 替换原文件
            os.remove(file_path)
            os.rename(temp_file_path, file_path)

# 指定文件夹路径
folder_path = './new_CTU_dbs/'
add_create_database_statement(folder_path)
